
## Imports

The above imports are not at all mandatory in order to have the following code work. This cell is here to ensure that your setup is correct. 

In [ ]:
%cd ..

%load_ext autoreload
%autoreload 2

## DDPG Agent on pendulum 


In [68]:
import acme

from src.environments.inverted_pendulum import InvertedPendulumEnv, inverted_pendulum_env_factory
from src.agents import DDPGAgent
from src.interaction_loops import ddpg_parallel_interaction_loop, evaluation_parallel_interaction_loop

In [69]:
pendulum_environment = InvertedPendulumEnv(for_evaluation=True)  #### WARNING, I changed the env
pendulum_environment_spec = acme.make_environment_spec(pendulum_environment)

In [72]:
# define the agent
num_evaluation_steps = 1000
max_learner_steps=100
pendulum_agent = DDPGAgent(seed=0, 
learning_rate=5e-4,
gamma=.95, 
environment_spec=pendulum_environment_spec, tau=1)

In [ ]:
D

In [73]:
ddpg_parallel_interaction_loop(pendulum_agent, env_factory=inverted_pendulum_env_factory, max_learner_steps=max_learner_steps, num_actors=16, batch_size=32, buffer_size=1000)


iteration nb10
actions_mean: 0.87	|mean_reward: -7.80	|obs: 0.37	|policy_loss: 3.11	|value_loss: 15.11	|value_mean: -3.11	|value_target_mean: -7.80
iteration nb20
actions_mean: 0.53	|mean_reward: -5.32	|obs: 0.15	|policy_loss: 6.68	|value_loss: 2.15	|value_mean: -6.69	|value_target_mean: -5.32
iteration nb30
actions_mean: 0.59	|mean_reward: -5.58	|obs: -0.02	|policy_loss: 4.04	|value_loss: 2.34	|value_mean: -4.04	|value_target_mean: -5.58
iteration nb40
actions_mean: 0.53	|mean_reward: -6.90	|obs: 0.16	|policy_loss: 3.96	|value_loss: 4.82	|value_mean: -3.96	|value_target_mean: -6.90
iteration nb50
actions_mean: 0.54	|mean_reward: -7.40	|obs: 0.01	|policy_loss: 6.20	|value_loss: 1.44	|value_mean: -6.20	|value_target_mean: -7.40
iteration nb60
actions_mean: 0.47	|mean_reward: -7.19	|obs: 0.03	|policy_loss: 6.05	|value_loss: 1.43	|value_mean: -6.05	|value_target_mean: -7.19
iteration nb70
actions_mean: 0.32	|mean_reward: -7.38	|obs: 0.29	|policy_loss: 5.73	|value_loss: 1.78	|value_mean: -

In [ ]:
pendulum_agent._learner_state.params

In [ ]:
from jax.tree_util import tree_map

In [ ]:
tree_map(lambda x,y:x+y, pendulum_agent._learner_state.params, pendulum_agent._learner_state.params)

In [ ]:
for _ in range(10):
  ddpg_parallel_interaction_loop(pendulum_agent, env_factory=inverted_pendulum_env_factory, max_learner_steps=max_learner_steps, sequence_length=sequence_length, num_actors=16)
  plot_policy_on_pendulum(pendulum_agent, 50)
  evaluation_parallel_interaction_loop(pendulum_agent, env_factory=inverted_pendulum_env_factory, sequence_length=200, num_actors=4)

In [ ]:
from src.networks import ValueNetworkDDPG, PolicyNetworkDDPG, PolicyNetwork
from src.data import Transition
import jax.numpy as jnp
import jax
import haiku as hk

In [ ]:
from src.data import Transition
observation = pendulum_environment_spec.observations.generate_value()
action = pendulum_environment_spec.actions.generate_value()

tr = Transition(
    obs_tm1= observation[None],
    action_tm1= action[None],
    reward_t= jnp.zeros(1)[None],
    discount_t=jnp.zeros(1)[None],
    obs_t=observation[None],
    done=jnp.zeros(1)[None])

In [ ]:
jnp.concatenate([tr.obs_t, tr.action_tm1], axis=1)

In [ ]:
def f(transition):
    action = PolicyNetworkDDPG(environment_spec.actions)(transition.obs_tm1, True)
    return action

def f1(obs):
    return hk.BatchApply(PolicyNetworkDDPG(environment_spec.actions))(obs, True)
    
init_loss, apply_loss = hk.without_apply_rng(hk.transform_with_state(f))

In [ ]:
rng = jax.random.PRNGKey(seed=0)

In [ ]:
init_loss(rng,tr)

In [ ]:
tr.obs_tm1